In [ ]:
import pandas as pd
import numpy as np
!pip install pyswarms

In [ ]:
data = pd.read_csv('OnlineRetail 2.csv', encoding='ISO-8859-1')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)]
data.drop(columns = ['Country', 'CustomerID'], inplace = True)
data['TotalSales'] = data['Quantity'] * data['UnitPrice']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,TotalSales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,20.34


In [ ]:
product_sales = data.groupby('StockCode').agg({
    'Quantity': 'sum',
    'UnitPrice': 'mean',
    'TotalSales': 'sum'
}).sort_values('TotalSales', ascending=False)
top_n = 10
top_products = product_sales.head(top_n).reset_index()

In [ ]:
def simulate_quantity_change(base_price, base_quantity, price_adjustment, elasticity=-1.2):
    """
    Estimate quantity change using a price elasticity model.
    """
    new_price = base_price * (1 + price_adjustment)
    quantity_ratio = (new_price / base_price) ** elasticity
    return base_quantity * quantity_ratio

In [ ]:
from pyswarms.single import GlobalBestPSO

# Extract base prices and quantities
base_prices = top_products['UnitPrice'].values
base_quantities = top_products['Quantity'].values

def objective_function(particles):
    """
    Objective function for PSO: maximize revenue.
    Each particle represents price adjustments for top products.
    """
    n_particles = particles.shape[0]
    fitness = np.zeros(n_particles)
    for i in range(n_particles):
        adjustments = particles[i]
        revenue = 0
        for j in range(len(base_prices)):
            new_qty = simulate_quantity_change(
                base_prices[j],
                base_quantities[j],
                adjustments[j]
            )
            new_price = base_prices[j] * (1 + adjustments[j])
            revenue += new_qty * new_price
        # Negative revenue for minimization
        fitness[i] = -revenue
    return fitness

In [ ]:
# Define bounds for price adjustments (-50% to +50%)
bounds = (np.full(top_n, -0.5), np.full(top_n, 0.5))

# PSO options
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

# Initialize PSO
optimizer = GlobalBestPSO(n_particles=20, dimensions=top_n, options=options, bounds=bounds)

# Perform optimization
best_cost, best_position = optimizer.optimize(objective_function, iters=50)

# Optimal price adjustments
optimal_adjustments = best_position

2025-04-18 17:36:44,441 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|50/50, best_cost=-3.17e+6
2025-04-18 17:36:44,537 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -3172966.818076142, best pos: [ 0.04412193 -0.09062026 -0.17711433  0.02562025 -0.49986949 -0.39825647
 -0.40401602 -0.49746861 -0.05052496 -0.49162987]


In [ ]:
print("Optimal Price Adjustments for Top Products:")
for code, adj in zip(top_products['StockCode'], optimal_adjustments):
    print(f"Product {code}: {'+' if adj >= 0 else ''}{adj*100:.1f}%")

estimated_revenue = -best_cost
print(f"\nEstimated Maximum Revenue: £{estimated_revenue:,.2f}")

Optimal Price Adjustments for Top Products:
Product DOT: +4.4%
Product 22423: -9.1%
Product 23843: -17.7%
Product 85123A: +2.6%
Product 47566: -50.0%
Product 85099B: -39.8%
Product 23166: -40.4%
Product M: -49.7%
Product POST: -5.1%
Product 23084: -49.2%

Estimated Maximum Revenue: £3,172,966.82
